### Data Analysis
Now that you’ve successfully merged the two datasets and have tackled descriptive statistics, thus successfully completing an vital part of the project, it’s time to focus on the analysis of key variables.
This phase will include a more in-depth analysis of the data to uncover trends, correlations, and hidden insights. Start by analyzing the customers' age (and age intervals) and perform the same analysis for the properties’ prices. Ultimately, find out more about the relationship between age and price.

### Analyzing Age

1 - Calculate age at the time of purchase. The first step of the analysis is determining the age of customers at the time of sale. As there isn’t such a variable in the data, you must work out the customers’ ages from the information given in the data.

2 - Create age interval categories. After calculating the customers' ages, you must create age intervals to evaluate group behavior. Separate the age into 10 intervals of equal length, or otherwise create the following intervals:
            (19.0, 25.0], (25.0, 31.0], (31.0, 36.0], (36.0, 42.0], (42.0, 48.0], (48.0, 54.0], (54.0, 59.0], (59.0, 65.0], (65.0, 71.0], (71.0, 76.0]

3 - Break down by age intervals. Finally, using summary statistics, determine how many properties have been sold by age intervals.

### Analyzing Properties

To analyze the properties, you must perform similar steps to the age analysis. Create 10 bins for the price intervals to answer the quiz questions successfully. But remember that the correct number of bins may vary in real-world situations.

Relationship between Properties and Age

To conclude this data analysis part of the project, you must examine the relationship between the properties and age variable. Discover the covariance and correlation between the two variables.